In [ ]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

import numpy
import tflearn
import tensorflow
import random
import json
import pickle

with open("/content/sample_data/intents.json") as file:
    data = json.load(file)

skip the below step if it's loaded already

In [ ]:
    words = []
    labels = []
    docs_x = []
    docs_y = []

    for intent in data["intents"]:
        for pattern in intent["patterns"]:
            wrds = nltk.word_tokenize(pattern)
            words.extend(wrds)
            docs_x.append(wrds)
            docs_y.append(intent["tag"])

        if intent["tag"] not in labels:
            labels.append(intent["tag"])

    words = [stemmer.stem(w.lower()) for w in words if w != "?"]
    words = sorted(list(set(words)))

    labels = sorted(labels)

    training = []
    output = []

    out_empty = [0 for _ in range(len(labels))]

    for x, doc in enumerate(docs_x):
        bag = []

        wrds = [stemmer.stem(w.lower()) for w in doc]

        for w in words:
            if w in wrds:
                bag.append(1)
            else:
                bag.append(0)

        output_row = out_empty[:]
        output_row[labels.index(docs_y[x])] = 1

        training.append(bag)
        output.append(output_row)


    training = numpy.array(training)
    output = numpy.array(output)

    with open("data.pickle", "wb") as f:
        pickle.dump((words, labels, training, output), f)


use the below code it it's loaded already to avoid training again

In [ ]:
 with open("data.pickle", "rb") as f:
      words, labels, training, output = pickle.load(f)

skip the below step if the model is trianed already

In [ ]:
tensorflow.reset_default_graph()

net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)
#remove try block if there are any changes to intents.json file and indent properly

model.fit(training, output, n_epoch=2000, batch_size=8, show_metric=True)
model.save("model.tflearn")

Training Step: 7999  | total loss: 0.05807 | time: 0.015s
| Adam | epoch: 2000 | loss: 0.05807 - acc: 0.9977 -- iter: 24/28
Training Step: 8000  | total loss: 0.05651 | time: 0.019s
| Adam | epoch: 2000 | loss: 0.05651 - acc: 0.9979 -- iter: 28/28
--
INFO:tensorflow:/content/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


use the below step to avoid training again

In [ ]:
 model.load("model.tflearn")

INFO:tensorflow:Restoring parameters from /content/model.tflearn


In [ ]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
            
    return numpy.array(bag)


def chat():
    print("Start talking with the bot (type quit to stop)!")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break

        results = model.predict([bag_of_words(inp, words)])
        results_index = numpy.argmax(results)
        tag = labels[results_index]

        for tg in data["intents"]:
            if tg['tag'] == tag:
                responses = tg['responses']

        print(random.choice(responses))

chat()

Start talking with the bot (type quit to stop)!
You: that is so huge
Cookies are on the menu!
You: it is so big
Cookies are on the menu!
You: big it is
that's what she said
You: that is so big
Cookies are on the menu!
You: sooo hugeeee
Cookies are on the menu!
You: it is so big
that's what she said
You: whats your name
I'm sandeep!
You: name of yours bitch
I'm sandeep aka sandy.
You: shut th
Hello!
You: well well well
Hi there, how can I help?
You: go quit
Good to see you again!
You: quit
